In [1]:
# !pip install mysql-connector-python
import pandas as pd
import mysql.connector as connection
import os
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Reading files from Input_files folder

path = os.path.join(os.getcwd(), 'Input_files')

files = glob.glob(path + "/*.xlsx")    # created a list of files
print(f"There are {len(files)} files available in the given folder path.")

df = pd.DataFrame()
for file in files:
    data = pd.read_excel(file)
    df = df.append(data, ignore_index=True)
    df = df.where(pd.notnull(df), 'na')
    
print(df.shape)    

There are 2 files available in the given folder path.
(1000, 14)


In [3]:
# To create the Table structure as per the dataframe

mydb = connection.connect(host ="localhost", user ="root", password = "12345", database = "mydb1")
cursor = mydb.cursor()

# To check if the table structure is available or not
query=("SELECT count(*) FROM information_schema.TABLES WHERE (TABLE_SCHEMA='mydb1') AND (TABLE_NAME='attribute')")
cursor.execute(query)

if cursor.fetchone()[0]==1:
    print(f"inventory Table structure is already available.")
else:
    columns = []
    for col_name, col_dtype in df.dtypes.items():
        if col_dtype == 'int64':
            col_dtype = 'INT'
            #col_dtype = 'BIGINT'
        elif col_dtype == 'float64':
            col_dtype = 'FLOAT'
        else:
            col_dtype = 'VARCHAR(255)'
        columns.append(f'{col_name} {col_dtype}')
    
    # below line will ocnvert list of column names and their data types in string for insert query
    create_table_query = f"CREATE TABLE attribute ({', '.join(columns)})"
        
    try:
        cursor.execute(create_table_query)
        mydb.commit()
        print("TABLE SCHEMA is Created Successfully.")
        
    except Exception as e:           
        mydb.close()
        print(f"There is problem in creating table schema "+ str(e))


inventory Table structure is already available.


In [4]:
# To insert data into Mysql table

#mydb = connection.connect(host ="localhost", user ="root", password = "12345", database = "mydb1")
#cursor = mydb.cursor()

for index, row in df.iterrows():
    values = list(row.values)
    placeholders = ', '.join(['%s'] * len(values))
    insert_query = f"INSERT INTO attribute VALUES ({placeholders})"   
    
    cursor.execute(insert_query, values)
    mydb.commit()



In [5]:
# To fetch the records from Mysql Table

#mydb = connection.connect(host ="localhost", user ="root", password = "12345", database = "mydb1")
#cursor = mydb.cursor()

query = "Select * from attribute"
df = pd.read_sql(query, mydb)
mydb.close()

df.head()


,Dress_ID,Style,Price,Rating,Size,Season,NeckLine,SleeveLength,waiseline,Material,FabricType,Decoration,Patter_Type,Recommendation
0,1006032852,Sexy,Low,4.6,M,Summer,o-neck,sleevless,empire,na,chiffon,ruffles,animal,1
1,1212192089,Casual,Low,0.0,L,Summer,o-neck,Petal,natural,microfiber,na,ruffles,animal,0
2,1190380701,vintage,High,0.0,L,Automn,o-neck,full,natural,polyster,na,na,print,0
3,966005983,Brief,Average,4.6,L,Spring,o-neck,full,natural,silk,chiffon,embroidary,print,1
4,876339541,cute,Low,4.5,M,Summer,o-neck,butterfly,natural,chiffonfabric,chiffon,bow,dot,0
